Volume Confirmation: Volume should ideally confirm the price movement. If the price of a stock is rising, higher volume is generally considered positive, indicating increased buying pressure. Conversely, if the price is falling, higher volume suggests greater selling pressure.

Breakouts and Reversals: Volume analysis can be particularly useful when identifying breakouts and reversals. Breakouts occur when a stock's price moves above a significant resistance level on high volume, indicating a potential trend continuation. Reversals, on the other hand, often accompany sharp price movements accompanied by a surge in volume.

Volume Patterns: Traders often look for specific volume patterns to gain insights into market sentiment. For example, a steady increase in volume over time may indicate accumulation or distribution by institutional investors. Additionally, spikes in volume can indicate panic selling or buying frenzies, suggesting potential market extremes.

Volume Divergence: Divergence occurs when the price of a stock moves in one direction while volume moves in the opposite direction. For instance, if a stock is experiencing a significant upward price movement but volume is decreasing, it might signal a lack of conviction behind the rally, potentially leading to a reversal.

Volume Analysis with Indicators: Volume analysis can be combined with technical indicators to enhance trading decisions. For example, traders often use volume-based indicators such as on-balance volume (OBV) or volume-weighted average price (VWAP) to confirm or diverge from price-based indicators like moving averages or relative strength index (RSI).

In [2]:
import pandas as pd
import yfinance as yf
from datetime import date
import matplotlib.pyplot as plt
import numpy as np

In [3]:
stocksymbols = 'ITC.NS'
ticker = yf.Ticker(stocksymbols)
end = date.today()
start = "2020-01-01"
df = ticker.history(interval="1d",start=start,end=end)
df.index = df.index.strftime('%d-%m-%y')
df.index = pd.to_datetime(df.index, format='%d-%m-%y')
df = df.loc[:,['Open','High','Low','Close','Volume']]
df = df.round(2)
df.tail()

,Open,High,Low,Close,Volume
Date,,,,,
2023-05-22,421.95,425.75,419.1,424.75,15421680
2023-05-23,426.00,431.20,424.2,429.15,8732608
2023-05-24,428.30,434.75,427.0,433.50,11479706
2023-05-25,436.95,442.45,434.8,441.15,18204464
2023-05-26,443.40,444.75,439.5,443.60,12995706


In [4]:
def volume_confirmation(df):
    # Check if the price is rising or falling
    df['Price_Movement'] = df['Close'].diff().fillna(0)
    df['Volume_Confirmation'] = 'No_Volume_Confirmation'

    for i in range(1, len(df)):
        if df['Price_Movement'].iloc[i] > 0 and df['Volume'].iloc[i] > df['Volume'].iloc[i-1]:
            df['Volume_Confirmation'].iloc[i] = 'Positive_Volume_Confirmation'
        elif df['Price_Movement'].iloc[i] < 0 and df['Volume'].iloc[i] > df['Volume'].iloc[i-1]:
            df['Volume_Confirmation'].iloc[i] = 'Negative_Volume_Confirmation'

    # Remove temporary columns
    df = df.drop(['Price_Movement'], axis=1)
    df = df.fillna(0)
    return df


df1 = volume_confirmation(df)
df1

C:\Users\Harsh\AppData\Local\Temp\ipykernel_12212\3027691458.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume_Confirmation'].iloc[i] = 'Positive_Volume_Confirmation'
C:\Users\Harsh\AppData\Local\Temp\ipykernel_12212\3027691458.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume_Confirmation'].iloc[i] = 'Negative_Volume_Confirmation'


,Open,High,Low,Close,Volume,Volume_Confirmation
Date,,,,,,
2020-01-01,202.78,202.78,201.50,202.35,4208837,No_Volume_Confirmation
2020-01-02,202.44,204.78,202.35,203.84,8402979,Positive_Volume_Confirmation
2020-01-03,204.82,204.82,202.27,202.69,9284478,Negative_Volume_Confirmation
2020-01-06,201.84,202.52,199.72,199.80,7636617,No_Volume_Confirmation
2020-01-07,200.61,202.18,199.38,200.02,8416741,Positive_Volume_Confirmation
...,...,...,...,...,...,...
2023-05-22,421.95,425.75,419.10,424.75,15421680,No_Volume_Confirmation
2023-05-23,426.00,431.20,424.20,429.15,8732608,No_Volume_Confirmation
2023-05-24,428.30,434.75,427.00,433.50,11479706,Positive_Volume_Confirmation


In [5]:
def volume_analysis(df):
    # Calculate price changes and their direction
    df['Price_Change'] = df['Close'].diff()
    df['Price_Direction'] = df['Price_Change'].apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))

    # Identify breakouts
    df['Breakout'] = 0
    resistance_level = df['High'].rolling(window=5).max()
    breakout_mask = (df['Close'] > resistance_level) & (df['Volume'] > df['Volume'].shift())
    df.loc[breakout_mask, 'Breakout'] = 1

    # Identify reversals
    df['Reversal'] = 0
    sharp_price_change = df['Price_Change'].rolling(window=5).sum()
    reversal_mask = (df['Price_Direction'].diff() != 0) & (df['Volume'] > df['Volume'].shift())
    df.loc[reversal_mask, 'Reversal'] = 1

    # Remove temporary columns
    df = df.drop(['Price_Change', 'Price_Direction'], axis=1)
    df = df.fillna(0)
    return df
df2 = volume_analysis(df1)
df2

,Open,High,Low,Close,Volume,Volume_Confirmation,Breakout,Reversal
Date,,,,,,,,
2020-01-01,202.78,202.78,201.50,202.35,4208837,No_Volume_Confirmation,0,0
2020-01-02,202.44,204.78,202.35,203.84,8402979,Positive_Volume_Confirmation,0,1
2020-01-03,204.82,204.82,202.27,202.69,9284478,Negative_Volume_Confirmation,0,1
2020-01-06,201.84,202.52,199.72,199.80,7636617,No_Volume_Confirmation,0,0
2020-01-07,200.61,202.18,199.38,200.02,8416741,Positive_Volume_Confirmation,0,1
...,...,...,...,...,...,...,...,...
2023-05-22,421.95,425.75,419.10,424.75,15421680,No_Volume_Confirmation,0,0
2023-05-23,426.00,431.20,424.20,429.15,8732608,No_Volume_Confirmation,0,0
2023-05-24,428.30,434.75,427.00,433.50,11479706,Positive_Volume_Confirmation,0,0


In [6]:
def calculate_volume_patterns(df):
    # Calculate percentage change in volume
    df['Volume_Change'] = df['Volume'].pct_change()

    # Calculate moving average of volume change
    df['Volume_MA'] = df['Volume_Change'].rolling(window=5).mean()

    # Identify spikes in volume
    df['Volume_Spike'] = (df['Volume_Change'] > 2 * df['Volume_MA']).astype(int)

    # Identify steady increase in volume over time
    df['Volume_Increase'] = (df['Volume_Change'] > df['Volume_Change'].shift(1)).astype(int)
    df = df.fillna(0)
    return df

df3 = calculate_volume_patterns(df2)
print(df3)

              Open    High     Low   Close    Volume  \
Date                                                   
2020-01-01  202.78  202.78  201.50  202.35   4208837   
2020-01-02  202.44  204.78  202.35  203.84   8402979   
2020-01-03  204.82  204.82  202.27  202.69   9284478   
2020-01-06  201.84  202.52  199.72  199.80   7636617   
2020-01-07  200.61  202.18  199.38  200.02   8416741   
...            ...     ...     ...     ...       ...   
2023-05-22  421.95  425.75  419.10  424.75  15421680   
2023-05-23  426.00  431.20  424.20  429.15   8732608   
2023-05-24  428.30  434.75  427.00  433.50  11479706   
2023-05-25  436.95  442.45  434.80  441.15  18204464   
2023-05-26  443.40  444.75  439.50  443.60  12995706   

                     Volume_Confirmation  Breakout  Reversal  Volume_Change  \
Date                                                                          
2020-01-01        No_Volume_Confirmation         0         0       0.000000   
2020-01-02  Positive_Volume_Confir

In [8]:
def calculate_volume_divergence(df):
    df['Volume_Divergence'] = None

    for i in range(1, len(df)):
        if df['Close'][i] > df['Close'][i-1] and df['Volume'][i] < df['Volume'][i-1]:
            df['Volume_Divergence'][i] = 'Bearish_Divergence'
        elif df['Close'][i] < df['Close'][i-1] and df['Volume'][i] > df['Volume'][i-1]:
            df['Volume_Divergence'][i] = 'Bullish_Divergence'
        else:
            df['Volume_Divergence'][i] = 'No_Divergence'

    return df
df4 = calculate_volume_divergence(df3)
print(df4)

C:\Users\Harsh\AppData\Local\Temp\ipykernel_12212\3133322006.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume_Divergence'][i] = 'No_Divergence'
C:\Users\Harsh\AppData\Local\Temp\ipykernel_12212\3133322006.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume_Divergence'][i] = 'Bullish_Divergence'
C:\Users\Harsh\AppData\Local\Temp\ipykernel_12212\3133322006.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['

              Open    High     Low   Close    Volume  \
Date                                                   
2020-01-01  202.78  202.78  201.50  202.35   4208837   
2020-01-02  202.44  204.78  202.35  203.84   8402979   
2020-01-03  204.82  204.82  202.27  202.69   9284478   
2020-01-06  201.84  202.52  199.72  199.80   7636617   
2020-01-07  200.61  202.18  199.38  200.02   8416741   
...            ...     ...     ...     ...       ...   
2023-05-22  421.95  425.75  419.10  424.75  15421680   
2023-05-23  426.00  431.20  424.20  429.15   8732608   
2023-05-24  428.30  434.75  427.00  433.50  11479706   
2023-05-25  436.95  442.45  434.80  441.15  18204464   
2023-05-26  443.40  444.75  439.50  443.60  12995706   

                     Volume_Confirmation  Breakout  Reversal  Volume_Change  \
Date                                                                          
2020-01-01        No_Volume_Confirmation         0         0       0.000000   
2020-01-02  Positive_Volume_Confir